<a href="https://colab.research.google.com/github/chris-creditdesign/nature-careers-survey-4-fact-check/blob/main/careers_phd_survey_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nature PhD Survey #4

Combine country name with World Bank Region

In [1]:
import pandas as pd

### Load and extract migration data from Excel file

In [2]:
# Load the excel file, sheet 'Lables'
excel_file_path = './content/Nature Graduate Survey_Raw_Data_anonymised for publishing_forCR.xlsx'
df_labels = pd.read_excel(excel_file_path, sheet_name='Labels')

In [3]:
# Excract and rename the relevant columns
df_extracted = df_labels[[
    'Q6a. What is your country of origin / country you grew up in?',
    'Q6b. Are you studying in your country of origin / country you grew up in?',
    'Q7 Which country/region do you currently study in?'
]].copy()

df_extracted.rename(columns={
    'Q6a. What is your country of origin / country you grew up in?': 'q6_a',
    'Q6b. Are you studying in your country of origin / country you grew up in?': 'q6_b',
    'Q7 Which country/region do you currently study in?': 'q7'
}, inplace=True)

display(df_extracted.head())

,q6_a,q6_b,q7
0,Italy,No,Germany
1,Australia,Yes,NaN
2,Spain,No,Austria
3,China,Yes,NaN
4,China,No,United Kingdom


In [4]:
# Remove rows where
# q6_a is 'Other' or empty
# q6_b is 'No'
# q7 is 'Other' or empty
df_filtered = df_extracted[
    (df_extracted['q6_a'] != 'Other') &
    (df_extracted['q6_a'].notna()) &
    (df_extracted['q6_b'] == 'No') &
    (df_extracted['q7'] != 'Other') &
    (df_extracted['q7'].notna())
].copy()

df_filtered.tail()

,q6_a,q6_b,q7
3773,Brazil,No,Portugal
3777,Ethiopia,No,United Kingdom
3778,Zambia,No,South Africa
3779,Ireland,No,United Kingdom
3780,Germany,No,Sweden


In [5]:
print(f"Total number of migrating PhDs: {len(df_filtered)}")

Total number of migrating PhDs: 1224


### Load CSV file with World Bank Regions

In [7]:
df_regions = pd.read_csv('./content/world-regions-according-to-the-world-bank.csv')

In [8]:
df_regions.head()

,Entity,Code,Year,World regions according to WB
0,Afghanistan,AFG,2023,"Middle East, North Africa, Afghanistan and Pak..."
1,Aland Islands,ALA,2023,Europe and Central Asia (WB)
2,Albania,ALB,2023,Europe and Central Asia (WB)
3,Algeria,DZA,2023,"Middle East, North Africa, Afghanistan and Pak..."
4,American Samoa,ASM,2023,East Asia and Pacific (WB)


## Load the PHD migration data

## Update the origin and destination country names to match the World Bank conventions

Missmatched names:

* "Korea, South" = "South Korea"
* "Congo, Democratic Republic of the" = "Democratic Republic of Congo"
* "Myanmar (Burma)" = "Myanmar"
* "East Timor (Timor-Leste)" = "East Timor"
* "Czech Republic (Czechia)" = "Czechia"
* Other = N/A

In [9]:
df_filtered[df_filtered['q6_a'] == 'Congo, Democratic Republic of the']

,q6_a,q6_b,q7
601,"Congo, Democratic Republic of the",No,France
3350,"Congo, Democratic Republic of the",No,Japan
3703,"Congo, Democratic Republic of the",No,Burkina Faso


In [10]:
# Create a dictionary with the mapping of old to new country names
country_name_mapping = {
    "Korea, South": "South Korea",
    "Congo, Democratic Republic of the": "Democratic Republic of Congo",
    "Myanmar (Burma)": "Myanmar",
    "East Timor (Timor-Leste)": "East Timor",
    "Czech Republic (Czechia)": "Czechia"
}

# Replace the country names in the 'Entity' column of df_regions
df_filtered['q6_a'] = df_filtered['q6_a'].replace(country_name_mapping)
df_filtered['q7'] = df_filtered['q7'].replace(country_name_mapping)

In [11]:
# Confirm that 'Congo, Democratic Republic of the' has been renamed
# to be 'Democratic Republic of Congo'
print(f"Total 'Congo, Democratic Republic of the': {len(df_filtered[df_filtered['q6_a'] == 'Congo, Democratic Republic of the'])}")
print(f"Total 'Democratic Republic of Congo': {len(df_filtered[df_filtered['q6_a'] == 'Democratic Republic of Congo'])}")

Total 'Congo, Democratic Republic of the': 0
Total 'Democratic Republic of Congo': 3


In [12]:
df_filtered.head()

,q6_a,q6_b,q7
0,Italy,No,Germany
2,Spain,No,Austria
4,China,No,United Kingdom
5,Nigeria,No,United States
7,Ethiopia,No,Kenya


## Merge the PHD migration values with the World Bank data to add country codes and regions

In [13]:
# Rename columns in df_regions for clarity before merging
df_regions_origin = df_regions.rename(columns={'Entity': 'origin_country',
                                               'Code': 'origin_code',
                                               'World regions according to WB': 'origin_region'})

df_regions_destination = df_regions.rename(columns={'Entity': 'destination_country',
                                                    'Code': 'destination_code',
                                                    'World regions according to WB': 'destination_region'})


In [14]:
df_regions_origin.head()

,origin_country,origin_code,Year,origin_region
0,Afghanistan,AFG,2023,"Middle East, North Africa, Afghanistan and Pak..."
1,Aland Islands,ALA,2023,Europe and Central Asia (WB)
2,Albania,ALB,2023,Europe and Central Asia (WB)
3,Algeria,DZA,2023,"Middle East, North Africa, Afghanistan and Pak..."
4,American Samoa,ASM,2023,East Asia and Pacific (WB)


In [15]:
# Rename the columns in the PhD data to match
df_phd = df_filtered.rename(columns={'q6_a': 'origin_country',
                                      'q7': 'destination_country'})

In [16]:
df_phd.head()

,origin_country,q6_b,destination_country
0,Italy,No,Germany
2,Spain,No,Austria
4,China,No,United Kingdom
5,Nigeria,No,United States
7,Ethiopia,No,Kenya


In [17]:
# Merge df_phd with df_regions_origin to get origin country code and region
df_phd = pd.merge(df_phd, df_regions_origin[['origin_country', 'origin_code', 'origin_region']],
                  on='origin_country', how='left')

# Merge the result with df_regions_destination to get destination country code and region
df_phd = pd.merge(df_phd, df_regions_destination[['destination_country', 'destination_code', 'destination_region']],
                  on='destination_country', how='left')

In [18]:
df_phd.head()

,origin_country,q6_b,destination_country,origin_code,origin_region,destination_code,destination_region
0,Italy,No,Germany,ITA,Europe and Central Asia (WB),DEU,Europe and Central Asia (WB)
1,Spain,No,Austria,ESP,Europe and Central Asia (WB),AUT,Europe and Central Asia (WB)
2,China,No,United Kingdom,CHN,East Asia and Pacific (WB),GBR,Europe and Central Asia (WB)
3,Nigeria,No,United States,NGA,Sub-Saharan Africa (WB),USA,North America (WB)
4,Ethiopia,No,Kenya,ETH,Sub-Saharan Africa (WB),KEN,Sub-Saharan Africa (WB)


In [19]:
## Check for any missing columns
missing_values = df_phd.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
origin_country         0
q6_b                   0
destination_country    0
origin_code            0
origin_region          0
destination_code       0
destination_region     0
dtype: int64


In [20]:
print(f"Total number of rows in df_phd: {len(df_phd)}")

Total number of rows in df_phd: 1224


In [21]:
count_origin_europe = len(df_phd[df_phd["origin_region"] == "Europe and Central Asia (WB)"])
print(f"Total number of PhDs from Europe: {count_origin_europe}")

Total number of PhDs from Europe: 347


In [23]:
origin_region_counts = df_phd['origin_region'].value_counts()
origin_region_counts

origin_region
Europe and Central Asia (WB)                                347
East Asia and Pacific (WB)                                  195
Sub-Saharan Africa (WB)                                     191
South Asia (WB)                                             177
Latin America and Caribbean (WB)                            127
Middle East, North Africa, Afghanistan and Pakistan (WB)    125
North America (WB)                                           62
Name: count, dtype: int64

In [24]:
destination_region_counts = df_phd['destination_region'].value_counts()
destination_region_counts

destination_region
Europe and Central Asia (WB)                                647
North America (WB)                                          247
East Asia and Pacific (WB)                                  230
Sub-Saharan Africa (WB)                                      41
Latin America and Caribbean (WB)                             23
Middle East, North Africa, Afghanistan and Pakistan (WB)     20
South Asia (WB)                                              16
Name: count, dtype: int64

In [25]:
origin_country_counts = df_phd['origin_country'].value_counts()
origin_country_counts

origin_country
India            135
China             92
Nigeria           59
Italy             58
United States     46
                ... 
Somalia            1
Namibia            1
Albania            1
Turkmenistan       1
Uruguay            1
Name: count, Length: 121, dtype: int64

In [26]:
destination_country_counts = df_phd['destination_country'].value_counts()
destination_country_counts

destination_country
United States           203
Germany                 136
United Kingdom          108
Australia                63
China                    52
Spain                    50
Canada                   44
Netherlands              41
Switzerland              40
Japan                    34
France                   32
Sweden                   30
Belgium                  28
Austria                  27
Italy                    27
South Africa             24
Czechia                  20
Poland                   18
India                    16
Norway                   16
Thailand                 15
Taiwan                   15
Malaysia                 15
South Korea              15
Ireland                  14
Finland                  13
Portugal                 13
New Zealand              11
Brazil                   11
Hungary                  11
Denmark                   8
Singapore                 7
Turkey                    6
Israel                    6
Saudi Arabia              6


### Print file to CSV

In [27]:
# Print df_phd to a CSV file
df_phd.to_csv('./content/phd-migration-with-regions.csv', index=False)